In [1]:
import tensorflow as tf
tf.enable_eager_execution()

import pandas as pd
import numpy as np
import os, string, re, csv


# set seeds for reproducability
from tensorflow import set_random_seed
from numpy.random import seed
set_random_seed(2)
seed(1)

import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))


In [2]:
idx2char = pd.read_csv("vocab.csv", header=None)
idx2char = idx2char.to_numpy()
vocab = vocab = [x[1] for x in idx2char]

# Creating a mapping from unique characters to indices
char2idx = pd.read_csv("char2idx.csv", header=None)
char2idx = char2idx.to_numpy()

In [3]:
# Creating a mapping from unique characters to indices
char2idx = pd.read_csv("char2idx.csv", header=None)
char2idx = char2idx[0].to_dict()


# The maximum length sentence we want for a single input in characters
seq_length = 100

# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [4]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.CuDNNLSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
    return model

model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

Instructions for updating:
Colocations handled automatically by placer.


In [6]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)


model.compile(optimizer='adam', loss=loss)

# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints_2'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            23296     
_________________________________________________________________
cu_dnnlstm_1 (CuDNNLSTM)     (1, None, 1024)           5251072   
_________________________________________________________________
dense_1 (Dense)              (1, None, 91)             93275     
Total params: 5,367,643
Trainable params: 5,367,643
Non-trainable params: 0
_________________________________________________________________


In [7]:
dir(model)

['__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_activity_regularizer',
 '_add_inbound_node',
 '_add_unique_metric_name',
 '_add_variable_with_custom_getter',
 '_all_metrics_tensors',
 '_all_stateful_metrics_tensors',
 '_base_init',
 '_build_input_shape',
 '_cache_output_metric_attributes',
 '_call_and_compute_mask',
 '_call_convention',
 '_call_is_graph_friendly',
 '_check_trainable_weights_consistency',
 '_checkpoint_dependencies',
 '_checkpointable_saver',
 '_clear_losses',
 '_compute_output_and_mask_jointly',
 '_compute_previous_mask',
 '_dataset_iterator_cache',
 '_default_save_signature',
 '_deferred_dependencies',
 '_determine_call_convention',
 '_

In [8]:
model.get_config()

{'name': 'sequential_1',
 'layers': [{'class_name': 'Embedding',
   'config': {'name': 'embedding_1',
    'trainable': True,
    'batch_input_shape': (1, None),
    'dtype': 'float32',
    'input_dim': 91,
    'output_dim': 256,
    'embeddings_initializer': {'class_name': 'RandomUniform',
     'config': {'minval': -0.05,
      'maxval': 0.05,
      'seed': None,
      'dtype': 'float32'}},
    'embeddings_regularizer': None,
    'activity_regularizer': None,
    'embeddings_constraint': None,
    'mask_zero': False,
    'input_length': None}},
  {'class_name': 'CuDNNLSTM',
   'config': {'name': 'cu_dnnlstm_1',
    'trainable': True,
    'dtype': 'float32',
    'return_sequences': True,
    'return_state': False,
    'go_backwards': False,
    'stateful': True,
    'time_major': False,
    'units': 1024,
    'kernel_initializer': {'class_name': 'GlorotUniform',
     'config': {'seed': None, 'dtype': 'float32'}},
    'recurrent_initializer': {'class_name': 'GlorotUniform',
     'config'